In [116]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.webdriver import ActionChains
from time import sleep
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [ ]:
driver.get("https://www.betsson.com/")
driver.find_element(By.XPATH, "/html/body/obg-app-root/div/obg-m-core-layout/header/mat-toolbar/mat-toolbar-row[1]/div[2]/obg-app-m-account-control-container/a[1]").click()
sleep(1)

In [14]:
elem = driver.find_element(By.XPATH, '//*[@id="mat-input-3"]')
elem.send_keys("cc.darvin@gmail.com")
sleep(1)
elem = driver.find_element(By.XPATH, '//*[@id="mat-input-4"]')
elem.send_keys('PDwCC!86')
sleep(1)
elem = driver.find_element(By.XPATH, '//*[@id="login"]/obg-m-login-with-dialog/obg-m-dialog-layout/div/obg-m-login-container/obg-m-generic-login-container/form/div[2]/button')
elem.click()
sleep(5)

In [4]:
elem = driver.find_element(By.XPATH, '/html/body/obg-app-root/div/obg-m-core-layout/header/mat-toolbar/mat-toolbar-row[2]/nav/obg-menu/a[1]')
elem.click()

In [106]:
def scroll_down(selector, amount=10):
    elem = driver.find_element(By.CSS_SELECTOR, selector)
    scroll_origin = ScrollOrigin.from_element(elem)
    ActionChains(driver)\
            .scroll_from_origin(scroll_origin, 0, amount)\
            .perform()
            
        
def scroll_up(selector):
    scroll_down(selector, -10000)

In [132]:
def scraping_betsson(url):
    #driver.get(url)
    game = driver.find_element(By.CSS_SELECTOR, 'obg-m-event-header').text.replace('\n', ' ')
    bet_groups = []
    elems = driver.find_elements(By.CSS_SELECTOR, 'obg-m-event-market-group')
    for elem in elems:
        elem_group = elem.find_element(By.CSS_SELECTOR, '.obg-m-event-market-group-header-name.ng-star-inserted')

        bets = []
        elem_bets = elem.find_elements(By.CSS_SELECTOR, 'obg-selection-container')
        if len(elem_bets) == 0:
            elem_group.click()        
        
        for bet in elem_bets:
            try:
                bets.append({
                    'name': bet.find_element(By.CSS_SELECTOR, '.obg-selection-content-label-wrapper').text,
                    'odds': bet.find_element(By.CSS_SELECTOR, 'obg-numeric-change').text,
                })
            except:
                continue
        #group.send_keys(Keys.DOWN)
        scroll_down('.obg-m-sportsbook-layout-secondary-container ng-scrollbar', 50*len(elem_bets))
        bet_groups.append({
            'name': elem_group.text,
            'bit': bets,
        })
    # scrolling top
    scroll_up('.obg-m-sportsbook-layout-secondary-container ng-scrollbar')
    return bet_groups




In [133]:
data = scraping_betsson('https://www.betsson.com/pe/apuestas-deportivas/futbol/esoccer/batalla-de-futbol-8-minutos-de-juegonorw?eventId=f-lMx4CCS5-U6kaCU-iyyNUw&eti=1')

In [135]:
data

[{'name': 'Ganador del Partido',
  'bit': [{'name': 'Monaco (Gaga)', 'odds': '3.52'},
   {'name': 'Empate', 'odds': '3.52'},
   {'name': 'Olympique Lyonnais (Duka)', 'odds': '1.95'}]},
 {'name': 'Total de goles',
  'bit': [{'name': 'más de 1.5', 'odds': '1.50'},
   {'name': 'menos de 1.5', 'odds': '2.41'}]},
 {'name': 'Doble Oportunidad',
  'bit': [{'name': '1X', 'odds': '1.80'},
   {'name': '12', 'odds': '1.27'},
   {'name': 'X2', 'odds': '1.27'}]},
 {'name': 'Partido Hándicap Goles',
  'bit': [{'name': 'Monaco (Gaga)\n+0', 'odds': '2.60'},
   {'name': 'Olympique Lyonnais (Duka)\n+0', 'odds': '1.44'},
   {'name': 'Monaco (Gaga)\n+1', 'odds': '1.36'},
   {'name': 'Olympique Lyonnais (Duka)\n-1', 'odds': '2.90'}]}]

In [119]:
import redis

r = redis.Redis(
  host='redis-12622.c277.us-east-1-3.ec2.cloud.redislabs.com',
  port=12622,
  password='TxVYpEfg4DwZSjDxerOiWxNEhgIZouKa')

In [125]:
p = r.pubsub()
p.subscribe('betsson')
while True:
    message = p.get_message()
    if message:
        print(message)

{'type': 'subscribe', 'pattern': None, 'channel': b'betsson', 'data': 1}
{'type': 'message', 'pattern': None, 'channel': b'betsson', 'data': b'Hello World'}
{'type': 'message', 'pattern': None, 'channel': b'betsson', 'data': b'Hello World'}
{'type': 'message', 'pattern': None, 'channel': b'betsson', 'data': b'Hello World'}
{'type': 'message', 'pattern': None, 'channel': b'betsson', 'data': b'otro'}
{'type': 'message', 'pattern': None, 'channel': b'betsson', 'data': b'10'}


KeyboardInterrupt: 